In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../")
from component.message import cm

In [ ]:
from component.model.recipe import Recipe
from component.tile.custom_aoi_tile import AoiTile
from component.tile.dashboard_tile import DashboardTile
from component.widget.alert_state import AlertState

from pathlib import Path
import json
from component.scripts.plots import get_individual_charts
import ipyvuetify as v

In [ ]:
build_alert = AlertState()
build_alert

In [ ]:
build_alert

In [ ]:
recipe = Recipe()

In [ ]:
# We need the AoiTile so it can create the SeplanAoi and will 
# set the feature collection in the model
aoi_tile = AoiTile(recipe=recipe)

In [ ]:
recipe.load("/home/dguerrero/module_results/se.plan/recipes/Myanmar_example.json")

In [ ]:
dashboard_tile = DashboardTile(recipe=recipe)

In [ ]:
dashboard_tile.reset()
dashboard_tile.summary_stats = json.loads(Path("data/summary_stats.json").read_text())["results"]

In [ ]:
suitability_data = {area_name:data["suitability"] for area_name, data in dashboard_tile.summary_stats.items()}
# suitability_data

In [ ]:
dashboard_tile

# Test suitability chart

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [11]:
from component.scripts.plots import get_stacked_bars_chart, get_bars_chart, parse_layer_data
import json
from component.widget.suitability_table import get_summary_table
from pathlib import Path
import ipyvuetify as v

In [12]:
summary_stats = json.loads(Path("data/summary_stats.json").read_text())["results"]

In [13]:
from component.parameter.vis_params import gradient

In [14]:
palettes = {"green": ["#EBFAF2", "#66c2a4", "#006d2c"]}
# palettes = {"green": ["#98FB98", "#32CD32", "#006400"]}

no_data_color = ["#353535"]  # the color used for values filtered by constraints
gradient_palette = gradient(levels=5, palette=palettes["green"])
gradient_palette

['#ebfaf2', '#a8decb', '#66c2a4', '#339768', '#006d2c']

In [15]:
chart = get_stacked_bars_chart(summary_stats)
chart

EChartsWidget(option=Option(angleAxis=None, aria=None, axisPointer=None, backgroundColor='#1e1e1e00', brush=No…

In [29]:
v.theme.dark=not v.theme.dark

In [ ]:
chart.theme='light'

In [ ]:
chart.theme='light'
chart.renderer='svg'

In [ ]:
chart.theme='dark'
chart.renderer='svg'

In [ ]:
layer_id = "plantation_growth_rates"
layer_data = recipe.seplan.benefit_model.get_layer_data("plantation_growth_rates")
aoi_names, values, colors = parse_layer_data(summary_results, layer_id)

In [ ]:
get_bars_chart(
    aoi_names,
    [values],
    [colors],
    [layer_data["name"]],
    custom_color=True,
    show_legend=False,
)

In [ ]:
aoi_names, values, colors, series_names = [], [], [], []
for layer_id in recipe.seplan.cost_model.ids:

    layer_data = recipe.seplan.cost_model.get_layer_data(layer_id)

    aoi_name, value, color = parse_layer_data(summary_results, layer_id)
    aoi_names.append(aoi_name)
    values.append(value)
    colors.append(color)
    series_names.append(layer_data["name"])

chart = get_bars_chart(
    aoi_names[0], values, series_names, series_colors = ['#4CAF50', '#2196F3'
    ], custom_item_color=False, custom_item_colors=colors, bars_width=70
)
chart

In [ ]:
print(chart)

In [ ]:
chart

In [ ]:
from component.widget.dashboard_layer_panels import LayerFull

In [ ]:
layer_data = recipe.seplan.cost_model.get_layer_data(layer_id)

In [ ]:
LayerFull(layer_data, chart)

In [ ]:
from component.scripts.plots import parse_suitability

In [ ]:
from ipecharts.option import Option, Legend, Tooltip, XAxis, YAxis, Title
from ipecharts.option.series import Bar
from ipecharts.echarts import EChartsWidget



In [ ]:
region_names, level_names, series_data = parse_suitability(dashboard_tile.summary_stats)

In [ ]:
series_data

In [ ]:
bars = []
for series in series_data:
    bars.append(
        Bar(
            **{
                "type":"bar",
                "name":series['name'],
                "stack":series["stack"],
                "data":series["data"],
                "itemStyle": series['itemStyle']
            }
        )
    )
bars

In [ ]:
option = Option(
    **{
        # 'title':Title(**{
        #     'text': 'Forest Restoration Suitability by Region (Horizontal with Separate Unsuitable Land)'
        # }),
        "backgroundColor":"#1e1e1e00",
        'legend': Legend(**{
            'data': ['Very Low', 'Low', 'Medium', 'High', 'Very High', 'Unsuitable land']
        }),
        'yAxis': YAxis(**{
            'type': 'category',
            'data': ['MMR', 'Custom_MMR_Bago__E_', 'Custom_MMR_Kachin']
        }),
        'xAxis': XAxis(**{
            'type': 'value'
        })
    },
    series=bars, tooltip=Tooltip()
)

In [ ]:
EChartsWidget(option=option)

### Test stats 

In [ ]:
ee_features = recipe.seplan.aoi_model.get_ee_features()
ee_features

In [ ]:
from component.tile.dashboard_tile import DashboardTile
from component.widget.alert_state import AlertState

In [ ]:
alert = AlertState()
alert

In [ ]:
dashboard_tile = DashboardTile()

In [ ]:
dashboard_tile.build(recipe, alert)

In [ ]:
dashboard_tile

## Test statistics from recipe

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../")
from component.scripts.statistics import parse_theme_stats, get_summary_statistics
from component.model.recipe import Recipe
from component.scripts.seplan import reduce_constraints
import ee
ee.Initialize()

In [ ]:
# Load a previously created recipe
recipe = Recipe()
recipe.load_model()
recipe.load("/home/dguerrero/module_results/se.plan/recipes/test_cundinamarca.json")
recipe.seplan_aoi.aoi_model.set_object(); # we have to do this manually
aoi = recipe.seplan_aoi.aoi_model.feature_collection

In [ ]:
summary_stats = get_summary_statistics(recipe.seplan)
summary_stats

In [ ]:
parse_theme_stats(summary_stats)

## Calculate the image percent cover pixel area

In [ ]:
from component.scripts.statistics import get_image_percent_cover_pixelarea
from component.scripts.seplan import mask_image

In [ ]:
aoi = recipe.seplan.aoi_model.feature_collection

In [ ]:
asset_id = recipe.seplan.constraint_model.assets[0]
data_type = recipe.seplan.constraint_model.data_type[0]
maskout_values = recipe.seplan.constraint_model.values[0]

asset_id, data_type, maskout_values

In [ ]:
# maskout any image from the model

masked_image = mask_image(asset_id, data_type, maskout_values)

In [ ]:
get_image_percent_cover_pixelarea(masked_image, aoi, "perro").getInfo()

In [ ]:
from sepal_ui.mapping import SepalMap
import ee

ee.Initialize()

In [ ]:
map_ = SepalMap()
map_.addLayer(masked_image.unmask(0).randomVisualizer().clip(aoi))
map_

## Calculate the image sum on COSTS

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from component.scripts.statistics import get_image_sum
from component.scripts.seplan import reduce_constraints, asset_to_image

In [ ]:
aoi = recipe.seplan.aoi_model.feature_collection

In [ ]:
asset_id = recipe.seplan.cost_model.assets[0]
constraint_list = recipe.seplan.get_masked_constraints_list()
mask_out_areas = reduce_constraints(constraint_list)

In [ ]:
image = asset_to_image(asset_id)

In [ ]:
sum_img = image.updateMask(mask_out_areas).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=aoi,
    scale=100,
    maxPixels=1e13,
)
sum_img.values().get(0).getInfo()

In [ ]:
get_image_sum(image, aoi, mask_out_areas, "name").getInfo()

In [ ]:
area_ha = (
    ee.Image.pixelArea()
    .divide(10000)
    .reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=100,
        maxPixels=1e13,
    )
    .getInfo()
)

In [ ]:
map_.addLayer(constraint_list[0][0].clip(aoi), {}, "mask")

In [ ]:
map_ = SepalMap()
map_.addLayer(mask_out_areas.randomVisualizer().clip(aoi))
map_